In [1]:
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
from FairRanking.helpers import nDCG_cls, disparate_impact, calc_accuracy, calc_sens_loss, rND_torch, auc_estimator, group_pairwise_accuracy
from FairRanking.datasets.adult import Adult
from FairRanking.datasets.law import Law
from FairRanking.datasets.compas import Compas
from FairRanking.datasets.wiki import Wiki
from FairRanking.models.BaseDirectRanker import convert_data_to_tensors
from FairRanking.models.DirectRankerAdv import DirectRankerAdv
from torch.optim.lr_scheduler import StepLR
from FairRanking.writer import Writer

In [7]:
data = Law('Race')
#data = Adult()
#data = Compas()
#data = Wiki()
#(X_train, s_train, y_train), (X_val, s_val, y_val), (X_test, s_test, y_test) = data.get_data(convert_to_tensor=False)

In [8]:
X_train0, X_train1, s_train0, s_train1, y_train, X_val0, X_val1, s_val0, s_val1, y_val, X_test0, X_test1, s_test0, s_test1, y_test = convert_data_to_tensors(data)

{0.0, 1.0, 2.0, 5.0, 8.0, 9.0}
<class 'numpy.ndarray'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
{0.0, 1.0, 2.0, 5.0, 8.0, 9.0}
<class 'numpy.ndarray'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
{0.0, 1.0, 2.0, 5.0, 8.0, 9.0}
<class 'numpy.ndarray'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [9]:
#import numpy as np
def calc_rnd(model, X0, X1, s0, s1):
    zero_documents = torch.zeros(size=(X0.shape[0]+X1.shape[0], X0.shape[1]))
    X_test_combined = torch.cat((X0, X1), dim=0)
    shuffled = X_test_combined[torch.randperm(X_test_combined.size(0))]
    s_test_combined = torch.cat((s0, s1), dim=0)
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_combined, shuffled)
        s_test_combined = torch.argmax(s_test_combined, dim=1)
        return rND_torch(predictions, s_test_combined)


In [11]:
def first_phase(model, X_train0, X_train1, y_train, optimizer, loss_fn):
    y_pred_train = model(X_train0, X_train1)
    main_loss = loss_fn(y_train, y_pred_train)
    main_loss.backward(retain_graph=True)
    optimizer.step()
    optimizer.zero_grad()
    return main_loss, y_pred_train


def second_phase(model, X_train0, X_train1, s_train0, s_train1, optimizer, loss_fn, loss_threshold):
    sensitive_pred0, sensitive_pred1 = model.forward_2(X_train0, X_train1)
    sensitive_loss = loss_fn(torch.cat((sensitive_pred0, sensitive_pred1), dim=0), torch.cat((s_train0, s_train1), dim=0))
    #sensitive_loss = loss_fn(sensitive_pred0, s_train0)
    sensitive_loss.backward()
    #reversed_gradients = [p.grad * -1.0 for p in model.parameters() if p.grad is not None]
    #for p in model.named_parameters():
    #    if 'debias' in p[0]:
    #        continue
    #    else:
    #        if p[1].grad is not None:
    #            if sensitive_loss.item() < loss_threshold:
    #                p[1].grad *= -1
    #            else:
    #                p[1].grad = torch.zeros_like(p[1].grad, dtype=torch.float32)

    optimizer.step()
    optimizer.zero_grad()
    return sensitive_loss

def model_rel_evaluation(model, writer, epoch, loss_fn):
    model.eval()
    with torch.no_grad():
        y_train_pred = model(X_train0, X_train1)
        y_val_pred = model(X_val0, X_val1)
        di_train_score = disparate_impact(y_train_pred, s_train0, s_train1)
        train_acc = calc_accuracy(y_pred_train, y_train)
        val_acc = calc_accuracy(y_val_pred, y_val)
        di_val_score = disparate_impact(y_val_pred, s_val0, s_val1)
        auc_train = auc_estimator(y_pred_train, y_train)
        auc_val = auc_estimator(y_val_pred, y_val)
        rnd_train = calc_rnd(model, X_train0, X_train1, s_train0, s_train1)
        rnd_val = calc_rnd(model, X_val0, X_val1, s_val0, s_val1)
        ndcg_train = nDCG_cls(model, X_train0, X_train1, y_train, esti=False)
        ndcg_val = nDCG_cls(model, X_val0, X_val1, y_val, esti=False)
        gpa_train = group_pairwise_accuracy(y_pred_train, y_train, s_train0)
        gpa_val = group_pairwise_accuracy(y_val_pred, y_val, s_val0) 
        disparate_impact_losses = {'train_di': di_train_score, 'val_di': di_val_score}
        epoch_accuracies = {'train_acc': train_acc, 'val_acc': val_acc}
        epoch_auc = {'train_auc': auc_train, 'val_auc': auc_val}
        epoch_rnd = {'train_rnd': rnd_train, 'val_rnd': rnd_val}
        epoch_ndcg = {'train_ndcg': ndcg_train, 'val_ndcg': ndcg_val}
        epoch_gpa = {'train_gpa': gpa_train, 'val_ndcg': gpa_val}
        input = {
            'Accuracy': [epoch_accuracies, epoch],
            'Disparate Impact': [disparate_impact_losses, epoch],
            'AUC': [epoch_auc, epoch],
            'rND': [epoch_rnd, epoch],
            'NDCG': [epoch_ndcg, epoch],
            'GPA': [epoch_gpa, epoch]
            }
        return train_acc, di_train_score, val_acc, di_val_score

def model_evaluation(model, sensitive_loss, writer, writer_epoch):
    model.eval()
    with torch.no_grad():
        y_train_pred = model(X_train0, X_train1)
        y_val_pred = model(X_val0, X_val1)
        y_pred_train = model(X_train0, X_train1)
        di_train_score = disparate_impact(y_pred_train, s_train0, s_train1)
        di_val_score = disparate_impact(y_val_pred, s_val0, s_val1)
        sensitive_val_pred0, sensitive_val_pred1 = model.forward_2(X_val0, X_val1)
        sensitive_val_loss = calc_sens_loss(sensitive_val_pred0, sensitive_val_pred1, s_val0, s_val1, gamma=1.0)
        train_acc = calc_accuracy(y_pred_train, y_train)
        val_acc = calc_accuracy(y_val_pred, y_val)
        auc_train = auc_estimator(y_pred_train, y_train)
        auc_val = auc_estimator(y_val_pred, y_val)
        rnd_train = calc_rnd(model, X_train0, X_train1, s_train0, s_train1)
        rnd_val = calc_rnd(model, X_val0, X_val1, s_val0, s_val1)
        ndcg_train = nDCG_cls(model, X_train0, X_train1, y_train, esti=False)
        ndcg_val = nDCG_cls(model, X_val0, X_val1, y_val, esti=False)
        gpa_train = group_pairwise_accuracy(y_pred_train, y_train, s_train0)
        gpa_val = group_pairwise_accuracy(y_val_pred, y_val, s_val0) 
        sensitive_losses = {'train_sensitive': sensitive_loss, 'val_sensitive': sensitive_val_loss}
        disparate_impact_losses = {'train_di': di_train_score, 'val_di': di_val_score}
        epoch_accuracies = {'train_acc': train_acc, 'val_acc': val_acc}
        epoch_auc = {'train_auc': auc_train, 'val_auc': auc_val}
        epoch_rnd = {'train_rnd': rnd_train, 'val_rnd': rnd_val}
        epoch_ndcg = {'train_ndcg': ndcg_train, 'val_ndcg': ndcg_val}
        epoch_gpa = {'train_gpa': gpa_train, 'val_ndcg': gpa_val}
        input = {
            'Accuracy': [epoch_accuracies, epoch],
            'Sensitive Loss': [sensitive_losses, epoch],
            'Disparate Impact': [disparate_impact_losses, epoch],
            'AUC': [epoch_auc, epoch],
            'rND': [epoch_rnd, epoch],
            'NDCG': [epoch_ndcg, epoch],
            'GPA': [epoch_gpa, epoch]
            }
        writer.write(**input)
        return train_acc, di_train_score, val_acc, sensitive_val_loss, di_val_score


def only_adversarial(model, X_train0, X_train1, s_train0, s_train1, optimizer, loss_fn):
    sensitive_pred0, sensitive_pred1 = model.forward_2(X_train0, X_train1)
    sensitive_loss = loss_fn(torch.cat((sensitive_pred0, sensitive_pred1), dim=0), torch.cat((s_train0, s_train1), dim=0))
    sensitive_loss.backward()
    for p in model.named_parameters():
        if 'debias' in p[0]:
            #print(p[1])
            continue
        else:
            if p[1].grad is not None:
                p[1].grad = torch.zeros_like(p[1].grad, dtype=torch.float32)

    optimizer.step()
    optimizer.zero_grad()
    return sensitive_loss.item()

In [12]:
scheduler_run = 'Schedule02'
# rnd score in the original data
y_test_full = torch.cat((y_test, (-1)*y_test), dim=0)
s_test_full = torch.cat((s_test0, s_test1), dim=0)
base_rnd = rND_torch(y_test_full, torch.argmax(s_test_full, dim=1))

hidden_layers = [64, 32, 16]
debias_layers = [64, 32, 16]
threshold = 0.0
main_lr = 0.001
adv_lr = 0.001
k = 1
schedules = [[i,j] for j in range(1,8) for i in range(1,8)]
schedules = [[1,1]]
for schedule in schedules:
    torch.manual_seed(42)
    model = DirectRankerAdv(num_features=X_train0.shape[1],
                     kernel_initializer=nn.init.normal_,
                     hidden_layers=hidden_layers,
                     bias_layers=debias_layers,
             )
    n_epochs = int(1000 / schedule[0])
    """writer = Writer(hidden_layers,
                    schedule,
                    scheduler_run,
                    'adult',
                    f'run_{k}',
                    'directRankerADV',
                    'disparateImpactNormalWeightInitial'
                    )
    writer.writer.add_text(tag='Debias Layers', text_string=str(debias_layers))
    writer.writer.add_text(tag='Main lr', text_string=str(main_lr))
    writer.writer.add_text(tag='Adv lr', text_string=str(adv_lr))
    writer.writer.add_text(tag='threshold', text_string=str(threshold))"""
    PATH = f'Results/{model.name}/{data.name}/{scheduler_run}/'
    lr_decay = 0.944
    optimizer = optim.Adam(model.parameters(), lr=main_lr)
    adv_optimizer = optim.Adam(model.parameters(), lr=adv_lr)
    opt_scheduler = StepLR(optimizer, step_size=500, gamma=lr_decay)
    adv_scheduler = StepLR(adv_optimizer, step_size=500, gamma=lr_decay)
    loss_fn = nn.MSELoss(reduction='mean')
    sensitive_loss_fn = nn.BCEWithLogitsLoss()
    writer_epoch = 0 
    for epoch in range(n_epochs):
        model.train()
        for i in range(schedule[0]):
            main_loss, y_pred_train = first_phase(model, X_train0, X_train1, y_train, optimizer, loss_fn)
            #model.eval()
            train_loss = main_loss.item()
            #train_acc, di_train_score, val_acc, di_val_score = \
            #    model_rel_evaluation(model, writer, epoch, loss_fn)
            model.train()
            writer_epoch += 1
        for i in range(schedule[1]):
            sensitive_loss = second_phase(model, X_train0, X_train1, s_train0, s_train1, adv_optimizer, sensitive_loss_fn, threshold)
            if epoch % 50 == 0:
                print(sensitive_loss)
            #with open('sensitive_loss.txt', 'a') as file:
            #    file.write(f"{writer_epoch}    {sensitive_loss.item():.4f}\n")
            model.eval()
            #train_acc, di_train_score, val_acc, sensitive_val_loss, di_val_score = \
            #    model_evaluation(model, sensitive_loss, writer, writer_epoch)
            writer_epoch += 1
            model.train()
    model.eval()
    with torch.no_grad():
        y_test_pred = model(X_test0, X_test1)
        test_loss = loss_fn(y_test, y_test_pred)
        test_acc = calc_accuracy(y_test_pred, y_test)
        di_test_score = disparate_impact(y_test_pred, s_test0, s_test1)
        print(f'Test Loss: {test_loss.item():.4f}\t Test Accuracy: {test_acc.item():.4f}\t DI: {di_test_score:.4f}')
        auc_test = auc_estimator(y_test_pred, y_test)
        ndcg_test = nDCG_cls(model, X_test0, X_test1, y_test, esti=False)
        gpa_test = group_pairwise_accuracy(y_test_pred, y_test, s_test0)
        sensitive_pred0, sensitive_pred1 = model.forward_2(X_test0, X_test1)
        sensitive_loss = sensitive_loss_fn(torch.cat((sensitive_pred0, sensitive_pred1), dim=0), torch.cat((s_test0, s_test1), dim=0))
        print(f'test sens loss: {sensitive_loss}')
        rnd_arr = []
        for i in range(10):
            rnd_arr.append(calc_rnd(model, X_test0, X_test1, s_test0, s_test1))
        rnd_test = np.mean(rnd_arr)
        with open(f'{PATH}results.txt', 'a+') as file:
            file.write(f'{auc_test},{test_acc.item()},{rnd_test},{ndcg_test},{gpa_test},{sensitive_loss},{str(schedule).replace(",",";")}\n')
    writer_epoch = 0
    """for epoch in range(n_epochs):
        for i in range(schedule[1]):
            model.train()
            sensitive_loss = only_adversarial(model, X_train0, X_train1, s_train0, s_train1, adv_optimizer, sensitive_loss_fn)
            if epoch % 50 == 0:
                print(sensitive_loss)
            #writer.write({'Train Sensitive Loss': sensitive_loss})
            #writer_epoch += 1
    model.eval()
    with torch.no_grad():
        sensitive_pred0, sensitive_pred1 = model.forward_2(X_test0, X_test1)
        test_sensitive_loss = sensitive_loss_fn(torch.cat((sensitive_pred0, sensitive_pred1), dim=0), torch.cat((s_test0, s_test1), dim=0))
        #writer.writer.add_text(tag='Sensitive Loss (Seperate Training)', text_string=str(test_sensitive_loss))
        with open(f'{PATH}results_extra_sensitive_training.txt', 'a+') as file:
            file.write(f'{str(str(schedule).replace(",",";"))},{test_sensitive_loss}\n')
    #del writer"""
    k += 1
    print('Finished')
print("ALL FINISHED!")

tensor(0.7674, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6348, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6040, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.5824, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.5303, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.5245, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.4911, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.4039, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.3498, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.2594, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.2168, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1994, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1812, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1749, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1673, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.1690, grad_fn=<BinaryCrossEntro

FileNotFoundError: [Errno 2] No such file or directory: 'Results/DirectRankerAdv/Law-race/Schedule02/results.txt'

In [25]:
from FairRanking.helpers import nDCG_cls_no_model
pred = model(X_train0, torch.zeros_like(X_train0))
_, sorted_pred_indices = torch.sort(pred, dim=0, stable=True)
_, sorted_y_indices = torch.sort(y_train, dim=0, stable=True)
print('AUC', auc_estimator(pred, y_train))
#print(torch.unique((pred == 0).int() == dict['y_train'], return_counts=True))
#print(nDCG_cls_no_model(pred, y_train))
print('rND:', rND_torch(pred, torch.argmax(s_train0, dim=1)))
# get the counts of correct and falsy ordered for accuracy
unique, counts = torch.unique(X_train0[sorted_pred_indices] == X_train0[sorted_y_indices], return_counts=True)
diff = abs(sorted_pred_indices - sorted_y_indices)
print(torch.mean(torch.tensor(diff, dtype=torch.float32)))
#print(torch.mean(diff))
print(unique, counts)
print('Accuracy:', counts[1]/(counts[0]+counts[1]))

tensor([ 1.,  1.,  1.,  ..., -1., -1., -1.])
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 

/var/folders/nx/wz1vqyld77ncdjz679wgbm_m0000gn/T/ipykernel_5240/1168841872.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(torch.mean(torch.tensor(diff, dtype=torch.float32)))


In [13]:
for p in model.named_parameters():
    print(p[0])

feature_layers.0.weight
feature_layers.0.bias
feature_layers.1.weight
feature_layers.1.bias
feature_layers.2.weight
feature_layers.2.bias
debias_layers.0.weight
debias_layers.0.bias
debias_layers.1.weight
debias_layers.1.bias
debias_layers.2.weight
debias_layers.2.bias
debias_layers.3.weight
debias_layers.3.bias
ranking_layer.weight


In [9]:
torch.manual_seed(42)
model = DirectRankerAdv(num_features=X_train0.shape[1],
                    kernel_initializer=nn.init.normal_,
                    hidden_layers=hidden_layers,
                    bias_layers=debias_layers,
            )
lr_decay = 0.944
optimizer = optim.Adam(model.parameters(), lr=main_lr)
adv_optimizer = optim.Adam(model.parameters(), lr=adv_lr)
opt_scheduler = StepLR(optimizer, step_size=500, gamma=lr_decay)
adv_scheduler = StepLR(adv_optimizer, step_size=500, gamma=lr_decay)
loss_fn = nn.MSELoss(reduction='mean')
sensitive_loss_fn = nn.BCEWithLogitsLoss()
writer_epoch = 0 
for epoch in range(n_epochs):
    model.train()
    # Main
    main_loss, y_pred_train = first_phase(model, X_train0, X_train1, y_train, optimizer, loss_fn)
    model.eval()
    train_loss = main_loss.item()
    model.train()
    writer_epoch += 1
    # Adversarial
    sensitive_loss = second_phase(model, X_train0, X_train1, s_train0, s_train1, adv_optimizer, sensitive_loss_fn, threshold)
    model.eval()
    writer_epoch += 1
    model.train()
    model.eval()
with torch.no_grad():
    y_test_pred = model(X_test0, X_test1)
    test_loss = loss_fn(y_test, y_test_pred)
    test_acc = calc_accuracy(y_test_pred, y_test)
    di_test_score = disparate_impact(y_test_pred, s_test0, s_test1)
    print(f'Test Loss: {test_loss.item():.4f}\t Test Accuracy: {test_acc.item():.4f}\t DI: {di_test_score:.4f}')
    auc_test = auc_estimator(y_test_pred, y_test)
    ndcg_test = nDCG_cls(model, X_test0, X_test1, y_test, esti=False)
    gpa_test = group_pairwise_accuracy(y_test_pred, y_test, s_test0)
    sensitive_pred0, sensitive_pred1 = model.forward_2(X_test0, X_test1)
    sensitive_loss = sensitive_loss_fn(torch.cat((sensitive_pred0, sensitive_pred1), dim=0), torch.cat((s_test0, s_test1), dim=0))
    rnd_arr = []
    for i in range(100):
        rnd_arr.append(calc_rnd(model, X_test0, X_test1, s_test0, s_test1))
    rnd_test = np.mean(rnd_arr)
    with open(f'{PATH}/results_base_ranker.txt', 'a+') as file:
        file.write(f'{auc_test},{test_acc.item()},{rnd_test},{ndcg_test},{gpa_test},{sensitive_loss},{str(schedule).replace(",",";")}\n')


KeyboardInterrupt: 